In [1]:
# The original data is provided by Naaijer et al. (2017). 
# Available at: https://github.com/MartijnNaaijer/phdthesis/blob/master/Various/subgenres_synvar.xls
# Notebook written with the assistance of Gemini 3 (January 7th 2026)

In [2]:
import pandas as pd
import xlrd

In [3]:
# Open the workbook with formatting info enabled
wb = xlrd.open_workbook('data/verses_genre/subgenres_verses_modified.xls', formatting_info=True)
sheet = wb.sheet_by_index(0)

In [4]:
# Define color mappings based on your legend
GENRE_MAP = {
    31: "prose",
    46: "prophetic",
    29: "poetry",
    50: "instruction",
    49: "list",
    55: "mixed",
    64: "standard"
}

In [5]:
# Define a function to retrieve the background color and font color indexes

def get_formatting(r, c):
    """Returns the background index and font index for a specific cell"""
    xf = wb.xf_list[sheet.cell_xf_index(r, c)]
    bg = xf.background.pattern_colour_index
    font = wb.font_list[xf.font_index].colour_index
    return bg, font

In [6]:
all_verses = []

In [7]:
# 3. Iterate through rows
for r in range(1, sheet.nrows):
    # Read values from the row
    bknr_raw = sheet.cell_value(r, 0)
    book_name = sheet.cell_value(r, 1)
    chnr_raw = sheet.cell_value(r, 2)
    
    # Get the "Chapter Genre" from the Book Name cell (Column 1)
    # This works row-by-row even if Chapter Num cells are empty
    chapter_bg_idx, _ = get_formatting(r, 1)
    chapter_genre = GENRE_MAP.get(chapter_bg_idx, "unknown")
    
    # 4. Iterate through verse columns starting at index 4 (Column E)
    for c in range(4, sheet.ncols):
        cell_val = sheet.cell_value(r, c)
        
        if cell_val == "" or cell_val is None:
            continue
            
        try:
            # Handle float to int conversion (e.g., 1.0 -> 1)
            v_num = int(float(cell_val))
            
            # Get specific verse formatting
            bg_idx, font_idx = get_formatting(r, c)
            
            # Language detection: Font Index 10 is Red (Aramaic)
            language = "Aramaic" if font_idx == 10 else "Hebrew"
            
            all_verses.append({
                "book_nr": int(float(bknr_raw)) if bknr_raw != "" else None,
                "book_name": book_name,
                "chapter_nr": int(float(chnr_raw)) if chnr_raw != "" else None,
                "chapter_genre": chapter_genre, 
                "verse_nr": v_num,
                "verse_genre": GENRE_MAP.get(bg_idx, "unknown"),
                "verse_language": language
            })
        except (ValueError, TypeError):
            continue

In [8]:
# Create DataFrame
df_final = pd.DataFrame(all_verses)

In [9]:
# Verify the results
print("--- Results Check (First 5 Rows) ---")
print(df_final.head(5))

--- Results Check (First 5 Rows) ---
   book_nr book_name  chapter_nr chapter_genre  verse_nr verse_genre  \
0        1   Genesis           1         prose         1       prose   
1        1   Genesis           1         prose         2       prose   
2        1   Genesis           1         prose         3       prose   
3        1   Genesis           1         prose         4       prose   
4        1   Genesis           1         prose         5       prose   

  verse_language  
0         Hebrew  
1         Hebrew  
2         Hebrew  
3         Hebrew  
4         Hebrew  


In [10]:
# Save the file
df_final.to_csv('data/verses_genre/verses_genre_structured_data.csv', index=False)